In [ ]:
import cv2
import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

import os
print(os.listdir("../input"))
import warnings
warnings.filterwarnings('ignore')
#%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from scipy.signal import argrelmin
warnings.simplefilter('ignore')
sns.set(rc={'figure.figsize' : (22, 10)})
sns.set_style("darkgrid", {'axes.grid' : True})

In [ ]:
def createKernel(kernelSize, sigma, theta):
    "create anisotropic filter kernel according to given parameters"
    assert kernelSize % 2 # must be odd size
    halfSize = kernelSize // 2

    kernel = np.zeros([kernelSize, kernelSize])
    sigmaX = sigma
    sigmaY = sigma * theta

    for i in range(kernelSize):
        for j in range(kernelSize):
            x = i - halfSize
            y = j - halfSize

            expTerm = np.exp(-x**2 / (2 * sigmaX) - y**2 / (2 * sigmaY))
            xTerm = (x**2 - sigmaX**2) / (2 * math.pi * sigmaX**5 * sigmaY)
            yTerm = (y**2 - sigmaY**2) / (2 * math.pi * sigmaY**5 * sigmaX)

            kernel[i, j] = (xTerm + yTerm) * expTerm

    kernel = kernel / np.sum(kernel)
    return kernel

In [ ]:
def crop_text_to_lines(text, blanks):
    x1 = 0
    y = 0
    lines = []
    for i, blank in enumerate(blanks):
        x2 = blank
        #print("x1=", x1, ", x2=", x2, ", Diff= ", x2-x1)
        line = text[:, x1:x2]
        lines.append((line,x1,x2))
        x1 = blank
    return lines
    

In [ ]:
def smooth(x, window_len=40, window='hanning'):
#     if x.ndim != 1:
#         raise ValueError("smooth only accepts 1 dimension arrays.") 
    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.") 
    if window_len<3:
        return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'") 
    s = np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w = np.ones(window_len,'d')
    else:
        w = eval('np.'+window+'(window_len)')

    y = np.convolve(w/w.sum(),s,mode='valid')
    return y

In [ ]:
def display_lines(lines_arr, orient='vertical'):
    plt.figure(figsize=(30, 30))
    if not orient in ['vertical', 'horizontal']:
        raise ValueError("Orientation is on of 'vertical', 'horizontal', defaul = 'vertical'") 
    if orient == 'vertical': 
        for i, l in enumerate(lines_arr):
            line = l[0]
            plt.subplot(5, 10, i+1)  # A grid of 2 rows x 10 columns
            plt.axis('off')
            plt.title("Line #{0}".format(i))
            _ = plt.imshow(line, cmap='gray', interpolation = 'bicubic')
            plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    else:
            for i, l in enumerate(lines_arr):
                line = l[0]
                plt.subplot(90, 1, i+1)  # A grid of 90 rows x 1 columns
                plt.axis('off')
                plt.title("Line #{0}".format(i))
                _ = plt.imshow(line, cmap='gray', interpolation = 'bicubic')
                plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()
    

In [ ]:
def applySummFunctin(img):
    res = np.sum(img, axis = 0)    #  summ elements in columns
    return res

In [ ]:
def normalize(img):
    (m, s) = cv2.meanStdDev(img)
    m = m[0][0]
    s = s[0][0]
    img = img - m
    img = img / s if s>0 else img
    return img

In [ ]:
def showImg(img, cmap=None):
    plt.imshow(img, cmap=cmap, interpolation = 'bicubic')
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()

In [ ]:
def transpose_lines(lines):
    res = []
    for l in lines:
        line = np.transpose(l[0])
        res.append((line,l[1],l[2]))
    return res

In [ ]:
def lineSegmentation(image, show=False, smoothness = 50, kernelSize=11, sigma=4, theta=1.5):
    #25, 0.8, 3.5
    #img = np.arange(16).reshape((4,4))
    imgFiltered1 = cv2.filter2D(image, -1, createKernel(kernelSize, sigma, theta), borderType=cv2.BORDER_REPLICATE)
    img4 = normalize(imgFiltered1)
    (m, s) = cv2.meanStdDev(imgFiltered1)
    summ = applySummFunctin(img4)
    #windows=['flat', 'hanning', 'hamming', 'bartlett', 'blackman']
    smoothed = smooth(summ, smoothness)
    mins = argrelmin(smoothed, order=2)
    arr_mins = np.array(mins)
    found_lines = crop_text_to_lines(img4, arr_mins[0])
    #sess = tf.Session()
    #found_lines_arr = []
    #with sess.as_default():
    #    for i in range(len(found_lines)-1):
    #        found_lines_arr.append(tf.expand_dims(found_lines[i], -1).eval())
    if show:
        res_lines = transpose_lines(found_lines)
        display_lines(res_lines, 'horizontal')
    return found_lines

In [ ]:
def wordSegmentation(image, show=False, smoothness = 45, kernelSize=19, sigma=9, theta=7, minArea=0):
    #img = np.arange(16).reshape((4,4))
    img = np.transpose(image)
    imgFiltered1 = cv2.filter2D(img, -1, createKernel(kernelSize, sigma, theta), borderType=cv2.BORDER_REPLICATE)
    img4 = normalize(imgFiltered1)
    (m, s) = cv2.meanStdDev(imgFiltered1)
    summ = applySummFunctin(img4)
    #windows=['flat', 'hanning', 'hamming', 'bartlett', 'blackman']
    smoothed = smooth(summ, smoothness)
    mins = argrelmin(smoothed, order=2)
    arr_mins = np.array(mins)
    found_lines = crop_text_to_lines(img4, arr_mins[0])
    #sess = tf.Session()
    found_lines_arr = []
    res_lines = []
    #with sess.as_default():
    for i in range(len(found_lines)-1):
        res_lines.append(found_lines[i])
         #found_lines_arr.append(tf.expand_dims(found_lines[i], -1).eval())
    if show:
        #res_lines = transpose_lines(found_lines)
        display_lines(res_lines)
    return res_lines

In [ ]:
def analyse_image(N_of_questions, filename):
    img = cv2.imread(filename)
    plt.imshow(img)
    img = cv2.resize(img, (1200, 1600))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    images = []
    i=0
    for im in lineSegmentation(img):
        j=0
        to_plot = []
        for word in wordSegmentation(im[0]):
            to_plot.append((word[0],0,0))
            images.append((word[0],word[1],im[1],word[2],im[2]))
            j+=1
            if j==3:
                #display_lines(to_plot)
                break
        if i==N_of_questions+1:
                break
        i+=1
    return images
    
analyse_image(10,'../input/test-liceum/IMG_20210106_171519.jpg')

In [ ]:
df = pd.read_csv('../input/encripyed-cyryllics-latin/ino.csv')

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import cv2



import torch.nn as nn
from torch.nn.functional import relu, sigmoid

class mach1(nn.Module):
    def __init__(self):
        super(mach1, self).__init__()
        self.conv0 = nn.Conv2d(3, 28, 3)
        self.fc0 = nn.ReLU()
        self.conv1 = nn.Conv2d(28, 14, 3)
        #self.drop1 = nn.Dropout2d(0.1)
        self.pool1 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(14*12*12,136)
        self.fc2 = nn.Linear(136,101)
        self.fc3 = nn.Linear(101,101)
        self.fc4 = nn.LogSoftmax(dim=1)
    def forward(self, x):
        x = self.conv0(x)
        x = self.fc0(x)
        x = self.conv1(x)
        #x = self.drop1(x)
        x = self.pool1(x)
        x = x.view(-1,14*12*12)
        x = sigmoid(self.fc1(x))
        x = sigmoid(self.fc2(x))
        x = sigmoid(self.fc3(x))
        x = self.fc4(x)
        return x



In [ ]:
model = mach1()

In [ ]:
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

def int_to_vector(x):
    answer = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                      0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    answer[x]=1
    return answer


X = df[["files"]]
y = df["ecription"].values#.apply(int_to_vector).values


In [ ]:
loss_fn = nn.CrossEntropyLoss()
#def loss_fn(x,y):
#    return -torch.dot(x.view(-1),y.view(-1))

In [ ]:
y

In [ ]:
losses = []
batch_size = X.shape[0]

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#SGD(model.parameters(), lr=0.42, momentum=0.9)


for e in range(2001):
    print(e,end=" ")
    for batch in range(X.shape[0]//batch_size):
        X_batch = []
        for filename in X.values[batch*batch_size:min((batch+1)*batch_size, X.shape[0])]:
            path = '../input/encripyed-cyryllics-latin/encdata/encdata/'+filename[0]
            img = cv2.imread(path)
            img = cv2.resize(img, (28,28))
            img = normalize(img)
            X_batch.append(np.rollaxis(img, axis=2, start=0))
            
        y_batch = y[batch*batch_size:min((batch+1)*batch_size, X.shape[0])] 
        X_train, X_test, y_train, y_test = train_test_split(X_batch, y_batch)
        
        X_train = Variable(torch.from_numpy(np.array(X_train)).float())
        X_test = Variable(torch.from_numpy(np.array(X_test)).float())
        y_train = Variable(torch.from_numpy(np.array(y_train.tolist())).float())
        y_test = Variable(torch.from_numpy(np.array(y_test.tolist())).float())
        
        out = model(X_train)
        y_train = y_train.long()
        loss = loss_fn(out,y_train)
        losses.append(loss.detach())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(e % 25 == 0):
            preds_test = model(X_test)
            y_test = y_test.long()
            loss_test = loss_fn(preds_test, y_test)
            s1 = torch.tensor([1]*len(y_test)).float()
            s2 = (torch.abs(y_test-torch.argmax(preds_test,dim=1))>0).float()
            print("acc:", torch.mean(s1-s2))
            print('Epoch:{0}, Error-Loss:{1}'.format(e, loss.item()))
            print('Epoch:{0}, Error-Test-Loss:{1}'.format(e , loss.item()))
            print('------------------------------------------------------')
        

In [ ]:
losses = []
batch_size = X.shape[0]

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#SGD(model.parameters(), lr=0.42, momentum=0.9)

for e in range(2001):
    print(e,end=" ")
    for batch in range(X.shape[0]//batch_size):
        X_batch = []
        for filename in X.values[batch*batch_size:min((batch+1)*batch_size, X.shape[0])]:
            path = '../input/encripyed-cyryllics-latin/encdata/encdata/'+filename[0]
            img = cv2.imread(path)
            img = cv2.resize(img, (28,28))
            img = normalize(img)
            X_batch.append(np.rollaxis(img, axis=2, start=0))
            
        y_batch = y[batch*batch_size:min((batch+1)*batch_size, X.shape[0])] 
        X_train, X_test, y_train, y_test = train_test_split(X_batch, y_batch)
        
        X_train = Variable(torch.from_numpy(np.array(X_train)).float())
        X_test = Variable(torch.from_numpy(np.array(X_test)).float())
        y_train = Variable(torch.from_numpy(np.array(y_train.tolist())).float())
        y_test = Variable(torch.from_numpy(np.array(y_test.tolist())).float())
        
        out = model(X_train)
        y_train = y_train.long()
        loss = loss_fn(out,y_train)
        losses.append(loss.detach())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(e % 25 == 0):
            preds_test = model(X_test)
            y_test = y_test.long()
            loss_test = loss_fn(preds_test, y_test)
            s1 = torch.tensor([1]*len(y_test)).float()
            s2 = (torch.abs(y_test-torch.argmax(preds_test,dim=1))>0).float()
            print("acc:", torch.mean(s1-s2))
            print('Epoch:{0}, Error-Loss:{1}'.format(e, loss.item()))
            print('Epoch:{0}, Error-Test-Loss:{1}'.format(e , loss.item()))
            print('------------------------------------------------------')
        

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
#SGD(model.parameters(), lr=0.42, momentum=0.9)


batch_size = X.shape[0]


for e in range(2001):
    print(e,end=" ")
    for batch in range(X.shape[0]//batch_size):
        X_batch = []
        for filename in X.values[batch*batch_size:min((batch+1)*batch_size, X.shape[0])]:
            path = '../input/encripyed-cyryllics-latin/encdata/encdata/'+filename[0]
            img = cv2.imread(path)
            img = cv2.resize(img, (28,28))
            img = normalize(img)
            X_batch.append(np.rollaxis(img, axis=2, start=0))
            
        y_batch = y[batch*batch_size:min((batch+1)*batch_size, X.shape[0])] 
        X_train, X_test, y_train, y_test = train_test_split(X_batch, y_batch)
        
        X_train = Variable(torch.from_numpy(np.array(X_train)).float())
        X_test = Variable(torch.from_numpy(np.array(X_test)).float())
        y_train = Variable(torch.from_numpy(np.array(y_train.tolist())).float())
        y_test = Variable(torch.from_numpy(np.array(y_test.tolist())).float())
        
        out = model(X_train)
        y_train = y_train.long()
        loss = loss_fn(out,y_train)
        losses.append(loss.detach())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(e % 25 == 0):
            preds_test = model(X_test)
            y_test = y_test.long()
            loss_test = loss_fn(preds_test, y_test)
            s1 = torch.tensor([1]*len(y_test)).float()
            s2 = (torch.abs(y_test-torch.argmax(preds_test,dim=1))>0).float()
            print("acc:", torch.mean(s1-s2))
            print('Epoch:{0}, Error-Loss:{1}'.format(e, loss.item()))
            print('Epoch:{0}, Error-Test-Loss:{1}'.format(e , loss.item()))
            print('------------------------------------------------------')
        

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
#SGD(model.parameters(), lr=0.42, momentum=0.9)

batch_size = X.shape[0]



for e in range(2001):
    print(e,end=" ")
    for batch in range(X.shape[0]//batch_size):
        X_batch = []
        for filename in X.values[batch*batch_size:min((batch+1)*batch_size, X.shape[0])]:
            path = '../input/encripyed-cyryllics-latin/encdata/encdata/'+filename[0]
            img = cv2.imread(path)
            img = cv2.resize(img, (28,28))
            img = normalize(img)
            X_batch.append(np.rollaxis(img, axis=2, start=0))
            
        y_batch = y[batch*batch_size:min((batch+1)*batch_size, X.shape[0])] 
        X_train, X_test, y_train, y_test = train_test_split(X_batch, y_batch)
        
        X_train = Variable(torch.from_numpy(np.array(X_train)).float())
        X_test = Variable(torch.from_numpy(np.array(X_test)).float())
        y_train = Variable(torch.from_numpy(np.array(y_train.tolist())).float())
        y_test = Variable(torch.from_numpy(np.array(y_test.tolist())).float())
        
        out = model(X_train)
        y_train = y_train.long()
        loss = loss_fn(out,y_train)
        losses.append(loss.detach())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(e % 25 == 0):
            preds_test = model(X_test)
            y_test = y_test.long()
            loss_test = loss_fn(preds_test, y_test)
            s1 = torch.tensor([1]*len(y_test)).float()
            s2 = (torch.abs(y_test-torch.argmax(preds_test,dim=1))>0).float()
            print("acc:", torch.mean(s1-s2))
            print('Epoch:{0}, Error-Loss:{1}'.format(e, loss.item()))
            print('Epoch:{0}, Error-Test-Loss:{1}'.format(e , loss.item()))
            print('------------------------------------------------------')
        

In [ ]:
model.parameters()

In [ ]:
torch.save(model.state_dict(), "model.d")
torch.save(model, "model.m")

In [ ]:
plt.plot(losses)
plt.show()

In [ ]:
decoder = {0:" ", 1:"1", 2:"2", 3:"3", 4:"4",
           5:"5", 6:"6", 7:"7", 8:"8", 9:"9",
          10:"а", 11:"б", 12:"в", 13:"г",
          14:"д", 15:"е", 16:"ж", 17:"з",
          18:"и", 19:"к", 20:"л", 21:"м",
          22:"н", 23:"о", 24:"п", 25:"р",
          26:"с", 27:"т", 28:"у", 29:"ф",
          30:"х", 31:"ц", 32:"ч", 33:"ш",
          34:"щ", 35:"ь", 36:"ы", 37:"ъ",
          38:"э", 39:"ю", 40:"я"}

def make_answer(num_of_questions, filename):
    image = cv2.imread(filename)
    image = cv2.resize(image, (1200, 1600))
    draw1 = image.copy()
    draw2 = image.copy()
    blocks = analyse_image(10,filename)
    i=0
    answer = []
    for seg in blocks:
        #print(seg)
        img = image[seg[1]:seg[3],seg[2]:seg[4],:]
        plt.imshow(img)
        plt.show()
        break
        draw1 = cv2.rectangle(draw1, (seg[4],seg[3]), (seg[2],seg[1]), (255,0,0), 3)       
        img = normalize(img)
        img = cv2.resize(img, (28,28))
        img = np.rollaxis(img, axis=2, start=0)
        img = torch.tensor([img]).float()
        answer.append(model(img).tolist()[0])
    res = torch.tensor(answer)
    counter = num_of_questions
    answer = {}
    while counter > 0:
        most_relevant = torch.argmax(torch.sum(torch.max(res)==res,dim=1))
        seg = blocks[most_relevant]
        draw2 = cv2.rectangle(draw2, (seg[3],seg[4]), (seg[1],seg[2]), (255,0,0), 3)
        answer[int(most_relevant)]=decoder[int(torch.argmax(res[most_relevant,:]))]
        res = torch.cat((res[:most_relevant,:], res[min(most_relevant+1,res.shape[0]):,:]), dim=0)
        counter -= 1
    plt.imshow(draw1)
    plt.show()
    plt.imshow(draw2)
    plt.show()
    return answer

filename = '../input/test-liceum/IMG_20210106_171526.jpg'
make_answer(10,filename)

In [ ]:
a = {2:"b",1:"a"}
print(a.keys())